In [1]:
import meshplot as mp

In [4]:
import numpy as np
import trimesh

from trimesh import proximity

In [5]:
mesh = trimesh.load('./data/monkey.obj')

print(f"Mesh loaded: {mesh.vertices.shape[0]} vertices, {mesh.faces.shape[0]} faces")

Mesh loaded: 2111 vertices, 3936 faces


In [157]:
import open3d as o3d

mesh = o3d.io.read_triangle_mesh('./data/monkey.obj')

amount = 5

# sample point cloud
# points = mesh.sample_points_poisson_disk(amount, 1.5)
points = mesh.sample_points_uniformly(amount)


In [158]:
hair = trimesh.load('./data/monkey_fur.obj')

# Group hair's vertices in block of 5
strands = np.array_split(hair.vertices, hair.vertices.shape[0] // 5)

strands_start = np.array([s[1:] for s in strands])
strands_end = np.array([s[:-1] for s in strands])
roots = np.array([s[0] for s in strands])

# Build a mesh from the strands
kd = trimesh.Trimesh(vertices=roots).kdtree
parents = [kd.query(p, k=3) for p in np.asarray(points.points)]

parents



[(array([0.00948285, 0.01236058, 0.02512565]), array([9047, 5593,  246])),
 (array([0.02077337, 0.02412071, 0.03970825]), array([1605, 5844, 8692])),
 (array([0.02008973, 0.03090559, 0.03415264]), array([7115, 2166, 6505])),
 (array([0.03253705, 0.03608607, 0.03632832]), array([2928,  942, 9445])),
 (array([0.00856741, 0.02627991, 0.0323695 ]), array([8703, 3785, 1579]))]

In [159]:
# render the trimesh with meshplot
p = mp.plot(np.asarray(mesh.vertices), np.asarray(mesh.triangles))
p.add_points(np.asarray(points.points), c='red', shading=dict(point_size=.01))

p.add_points(roots, c='green', shading=dict(point_size=.05))
p.add_lines(np.concatenate(strands_start), np.concatenate(strands_end), shading=dict(line_width=.005))

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0, 0.24…

3